In [20]:
import spotipy as spot
from spotipy.oauth2 import SpotifyClientCredentials
from collections import defaultdict
import networkx as nx
import matplotlib.pyplot as plt
import csv

In [21]:

cid = "0fe1efb5886e47f2b3b13d7efc1d6c24"
secret = "11f204ab8f7742b4bbba068f873747d4"
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)

sp = spot.Spotify(client_credentials_manager = client_credentials_manager)
unique_artists = set()
def create_artists_network(id):

    artists_network = {}
    artists = set()
    edges = set()
    connection_degree = {}

    artist_0 = sp.artist(id)
    artists_network["id"] = artist_0["id"]
    artists_network["name"] = artist_0["name"]
    artists_network["genres"] = artist_0["genres"]
    artists_network["popularity"] = artist_0["popularity"]
    artists_network["related_artists"] = {}

    artists.add(artist_0["name"])
    connection_degree[artist_0["name"]] = 0

    artists_1 = sp.artist_related_artists(id)

    for i in range(20):
        artists_network["related_artists"][i] = {}
        artists_network["related_artists"][i]["id"] = artists_1['artists'][i]['id']

        artists_network["related_artists"][i]["name"] = artists_1['artists'][i]['name']
        artists_network["related_artists"][i]["genres"] = artists_1['artists'][i]['genres']
        artists_network["related_artists"][i]["popularity"] = artists_1['artists'][i]['popularity']

        artists_2 = sp.artist_related_artists(artists_1['artists'][i]['id'])
        artists_network["related_artists"][i]["related_artists"] = {}

        artists.add(artists_1['artists'][i]['name'])
        edges.add(tuple(sorted(([artist_0["name"], artists_1['artists'][i]['name']]))))
        if artists_1['artists'][i]['name'] not in connection_degree:
            connection_degree[artists_1['artists'][i]['name']] = 1
        for j in range(19):
            artists_network["related_artists"][i]["related_artists"][j] = {}
            artists_network["related_artists"][i]["related_artists"][j]["id"] =artists_2['artists'][j]['id']
            artists_network["related_artists"][i]["related_artists"][j]["name"] = artists_2['artists'][j]['name']
            artists_network["related_artists"][i]["related_artists"][j]["genres"] = artists_2['artists'][j]['genres']
            artists_network["related_artists"][i]["related_artists"][j]["popularity"] = artists_2['artists'][j]['popularity']

            artists.add(artists_2['artists'][j]['name'])
            edges.add(tuple(sorted(([artists_1['artists'][i]['name'], artists_2['artists'][j]['name']]))))
            if artists_2['artists'][j]['name'] not in connection_degree:
                connection_degree[artists_2['artists'][j]['name']] = 2


    return artists_network, artists, edges

artists_network, artists, edges = create_artists_network("6jJ0s89eD6GaHleKKya26X")

In [22]:
with open('output.csv', 'w', newline='') as csvfile:
    fieldnames = ['x', 'y', 'value']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    
    for (x, y) in edges:
        writer.writerow({'x': x, 'y': y, 'value': 1})